<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_FeatureStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [8]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 56.8 MB/s eta 0:00:00


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os
import seaborn as sns; sns.set()
import polars as pl

# Configuration

In [3]:
class config:
    MAX_PROTEINS = 20
    PROFILE_REPORT = True
    
class paths:
    SAMPLE_SUBMISSION = "/content/drive/MyDrive/AMP/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = "/content/drive/MyDrive/AMP/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = "/content/drive/MyDrive/AMP/train_clinical_data.csv"
    TRAIN_PEPTIDES = "/content/drive/MyDrive/AMP/train_peptides.csv"
    TRAIN_PROTEINS = "/content/drive/MyDrive/AMP/train_proteins.csv"
    TEST_CLINICAL_DATA = "/content/drive/MyDrive/AMP/example_test_files/test.csv"
    TEST_PEPTIDES = "/content/drive/MyDrive/AMP/example_test_files/test_peptides.csv"
    TEST_PROTEINS = "/content/drive/MyDrive/AMP/example_test_files/test_proteins.csv"

# Function Definition

In [35]:
#統計量の作成
def make_statistic(df,group_col:str,agg_list:list):
    df_ret = df.groupby(group_col).agg([
        pl.mean(agg_list).suffix('_mean')
        ,pl.std(agg_list).suffix("_std")
        ,pl.median(agg_list).suffix("_med")
        #,pl.col(agg_list).max().suffix("_max") 
        #,pl.col(agg_list).min().suffix("_min") 
        #,pl.quantile(agg_list',0.25).suffix("_quan1")
        #,pl.quantile(agg_list',0.75).suffix("_quan3")
        ])
    return df_ret

# Load Data

In [91]:
train_clinical = pd.read_csv(paths.TRAIN_CLINICAL_DATA)
train_peptides = pd.read_csv(paths.TRAIN_PEPTIDES)
train_proteins = pd.read_csv(paths.TRAIN_PROTEINS)
supplemental_clinical = pd.read_csv(paths.SUPPLEMENTAL_CLINICAL_DATA)
#print('Correct label:Proteins shape: (232741, 5) Peptides shape: (981834, 6) Clinical shape: (2615, 8)')
print('Proteins shape:',train_proteins.shape)
print('Peptides shape:',train_peptides.shape)
print('Clinical shape:',train_clinical.shape)
print('supplemental_clinical shape:',supplemental_clinical.shape)

Proteins shape: (232741, 5)
Peptides shape: (981834, 6)
Clinical shape: (2615, 8)
supplemental_clinical shape: (2223, 8)


#Protain

In [48]:
train_proteins['UniProt'].value_counts()

P01024    1113
P05090    1113
P01011    1113
P01023    1113
Q92520    1113
          ... 
Q6UX71     661
P01780     654
Q562R1     616
Q99832     606
Q99829     489
Name: UniProt, Length: 227, dtype: int64

In [67]:
train_proteins_ = pl.DataFrame(train_proteins).select([
    pl.mean('NPX').over('visit_id').alias('NPX_mean_by_id')
   ,pl.std('NPX').over('visit_id').alias('NPX_std_by_id')
   ,pl.median('NPX').over('visit_id').alias('NPX_median_by_id')
   ,pl.mean('NPX').over('UniProt').alias('NPX_mean_by_Uni')
   ,pl.std('NPX').over('UniProt').alias('NPX_std_by_Uni')
   ,pl.median('NPX').over('UniProt').alias('NPX_median_by_Uni')
])
train_proteins_.head()



NPX_mean_by_id,NPX_std_by_id,NPX_median_by_id,NPX_mean_by_Uni,NPX_std_by_Uni,NPX_median_by_Uni
f64,f64,f64,f64,f64,f64
3.1805e6,2.6727e7,123757.0,11641.264435,2817.00353,11546.4
3.1805e6,2.6727e7,123757.0,511164.883903,235735.664694,483442.5
3.1805e6,2.6727e7,123757.0,26505.529157,10705.15254,26529.7
3.1805e6,2.6727e7,123757.0,27305.934884,8446.187506,26624.0
3.1805e6,2.6727e7,123757.0,17688.295406,7166.325369,17399.6
3.1805e6,2.6727e7,123757.0,3004.990691,1142.159575,2865.46
3.1805e6,2.6727e7,123757.0,126151.780054,72748.393517,116900.0
3.1805e6,2.6727e7,123757.0,50773.474638,21382.028764,50375.8
3.1805e6,2.6727e7,123757.0,195599.363694,79739.704279,185616.0


#Peptide

In [49]:
train_peptides['UniProt'].value_counts()

P02787    51916
P02768    40900
P01024    35253
P02751    27370
P02649    23082
          ...  
Q6UX71      661
P01780      654
Q562R1      616
Q99832      606
Q99829      489
Name: UniProt, Length: 227, dtype: int64

In [68]:
train_peptides_ = pl.DataFrame(train_peptides).select([
    pl.mean('PeptideAbundance').over('visit_id').alias('Abu_mean_by_id')
   ,pl.std('PeptideAbundance').over('visit_id').alias('Abu_std_by_id')
   ,pl.median('PeptideAbundance').over('visit_id').alias('Abu_median_by_id')
   ,pl.mean('PeptideAbundance').over('UniProt').alias('Abu_mean_by_Uni')
   ,pl.std('PeptideAbundance').over('UniProt').alias('Abu_std_by_Uni')
   ,pl.median('PeptideAbundance').over('UniProt').alias('Abu_median_by_Uni')
])
train_peptides_.head()

Abu_mean_by_id,Abu_std_by_id,Abu_median_by_id,Abu_mean_by_Uni,Abu_std_by_Uni,Abu_median_by_Uni
f64,f64,f64,f64,f64,f64
748153.907014,4.2318e6,93134.8,11641.264435,2817.00353,11546.4
748153.907014,4.2318e6,93134.8,68286.323527,57440.655716,47621.5
748153.907014,4.2318e6,93134.8,68286.323527,57440.655716,47621.5
748153.907014,4.2318e6,93134.8,68286.323527,57440.655716,47621.5
748153.907014,4.2318e6,93134.8,68286.323527,57440.655716,47621.5


#Clinical

In [ ]:
train_clinical['visit_month'].nunique()
train_clinical['visit_month'].unique()
train_clinical['visit_month'].value_counts()
#[  0,   3,   6,   9,  12,  18,  24,  30,  36,  42,  48,  54,  60, 72,  84,  96, 108] 17types
#⇒ Phase_1 初期 [0,3,6,9,12,18,24] Phase_2 中期 [30,36,42,48,54,60] Phase_3 後期[72,84,96,108]

In [128]:


train_clinical_pv=train_clinical.dropna(axis=0).pivot(index="patient_id",columns='visit_month',values='visit_id').reset_index()
train_clinical_pv.head()

#testデータ用の候補探し
test_patient_list = list(train_clinical_pv[
    ~train_clinical_pv[0].isna() & ~train_clinical_pv[6].isna() & ~train_clinical_pv[12].isna() & ~train_clinical_pv[24].isna()
]['patient_id'].values)
test_patient_list

[1517,
 3863,
 4923,
 7265,
 8699,
 10138,
 11459,
 14811,
 16566,
 17154,
 22623,
 27464,
 27893,
 29417,
 30119,
 30416,
 30951,
 35231,
 37312,
 37566,
 39719,
 40650,
 40876,
 40980,
 42086,
 45161,
 51846,
 51879,
 54979,
 57321,
 57416,
 57478,
 58597,
 62437,
 62732]

In [56]:
#欠損値処理
train_clinical.dropna(subset=['updrs_1'],inplace=True)
train_clinical.dropna(subset=['updrs_2'],inplace=True)

#upd23dの置き換え
train_clinical.replace({'On':1,'Off':2},inplace=True)

train_clinical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2613 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2613 non-null   object 
 1   patient_id                           2613 non-null   int64  
 2   visit_month                          2613 non-null   int64  
 3   updrs_1                              2613 non-null   float64
 4   updrs_2                              2613 non-null   float64
 5   updrs_3                              2588 non-null   float64
 6   updrs_4                              1576 non-null   float64
 7   upd23b_clinical_state_on_medication  1288 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 183.7+ KB


In [69]:
train_clinical_ = pl.DataFrame(train_clinical).groupby('visit_id').agg([
    pl.mean('updrs_1').suffix('_mean')
    ,pl.std('updrs_1').suffix("_std")
    ,pl.median('updrs_1').suffix("_med")
    ])

train_clinical_.head()

visit_id,updrs_1_mean,updrs_1_std,updrs_1_med
str,f64,f64,f64
"""41444_36""",3.0,0.0,3.0
"""11459_6""",15.0,0.0,15.0
"""16778_18""",4.0,0.0,4.0
"""50611_12""",11.0,0.0,11.0
"""35465_3""",5.0,0.0,5.0


In [119]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html


for i in range(3):
  train_clinical['rolling_updrs_{0}'.format(i+1)] = train_clinical.groupby('patient_id')['updrs_{0}'.format(i+1)].rolling(3,min_periods=1).mean().reset_index(drop=True)
  train_clinical['ewm_updrs_{0}'.format(i+1)] = train_clinical.groupby('patient_id')['updrs_{0}'.format(i+1)].transform(lambda x: x.ewm(halflife=10).mean()).reset_index(drop=True)
train_clinical.head()


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,rolling_updrs_1,ewm_updrs_1,rolling_updrs_2,ewm_updrs_2,rolling_updrs_3,ewm_updrs_3
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,10.000000,10.000000,6.000000,6.000000,15.000000,15.000000
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,10.000000,10.000000,6.500000,6.517322,20.000000,20.173217
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,9.333333,9.286627,7.666667,7.759546,24.666667,25.105042
3,55_9,55,9,8.0,9.0,30.0,0.0,On,8.666667,8.930796,8.666667,8.102607,29.666667,26.458798
4,55_12,55,12,10.0,10.0,41.0,0.0,On,8.666667,9.175259,9.666667,8.536426,35.000000,29.783493


#ProPep

In [23]:
train_ProPep = train_peptides.merge(train_proteins[['visit_id','UniProt','NPX']],on=['visit_id','UniProt'],how='left')
print(train_ProPep.shape)
train_ProPep.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [ ]:
train_ProPep_agg = train_ProPep.groupby('visit_id').mean()
train_all = train_clinical.merge(train_ProPep_agg[['PeptideAbundance','NPX']],on='visit_id',how='left')
print('shape',train_all.shape)
train_all.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,NPX
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0,732430.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0,732430.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9,732430.0
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7,732430.0
...,...,...,...,...,...,...,...
645363,21126_36,36,21126,P10909,EILSVDC(UniMod_4)STNNPSQAK,251923.0,8338150.0
645364,21126_36,36,21126,P10909,EIQNAVNGVK,152618.0,8338150.0
645365,21126_36,36,21126,P10909,FM(UniMod_35)ETVAEK,86861.3,8338150.0
645366,21126_36,36,21126,P10909,FMETVAEK,1581860.0,8338150.0


#Train All

In [ ]:
df_train = train_all[['visit_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4','PeptideAbundance','NPX']]
df_train.head()
#df_train.info()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,PeptideAbundance,NPX
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,748153.907014,2.028789e+07
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,685218.599872,1.843914e+07
3,55_9,55,9,8.0,9.0,30.0,0.0,On,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,On,740535.423563,2.010976e+07
...,...,...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0,Off,NaN,NaN
2611,65043_54,65043,54,4.0,8.0,11.0,1.0,Off,NaN,NaN
2612,65043_60,65043,60,6.0,6.0,16.0,1.0,Off,NaN,NaN
2613,65043_72,65043,72,3.0,9.0,14.0,1.0,Off,NaN,NaN


In [ ]:
#1行だけ欠損を処理
df_train=df_train.dropna(subset=['updrs_1'])

X = df_train.drop(['visit_id','updrs_1'],axis=1).values
y = df_train['updrs_1'].values
train_column_list = df_train.drop(['visit_id','updrs_1'],axis=1).columns

#X = df_train.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1).values
#y = df_train['updrs_{0}'.format(i + 1)].values